# Getting Started with AgentRun CLI

## Overview

In this tutorial, you will learn how to use **AgentRun CLI** to develop, package, build, and deploy AI agents to AgentCube. AgentRun CLI is a comprehensive developer tool that streamlines the complete agent lifecycle from local development to cloud deployment.

We will walk through a practical example using a **LangChain Agent** that provides tool-calling capabilities with Calculator and Weather tools via FastAPI.

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Practical/Hands-on                                                           |
| Agent type          | LangChain-based API Service                                                  |
| Framework           | LangChain + FastAPI                                                          |
| Language            | Python 3.8+                                                                  |
| Tutorial components | Local development, packaging, building, publishing, and invoking agents      |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Beginner-friendly                                                            |
| Tools used          | AgentRun CLI, Docker, LangChain                                              |

### What You'll Learn

* How to structure an AI agent project
* How to use AgentRun CLI commands (`pack`, `build`, `publish`, `invoke`, `status`)
* How to configure agent metadata
* How to build and deploy agents to AgentCube
* Best practices for agent development

### Tutorial Architecture

The complete workflow consists of five main stages:

1. **Development**: Create your agent code locally
2. **Packaging**: Use `agentrun pack` to prepare the workspace
3. **Building**: Use `agentrun build` to create container images
4. **Publishing**: Use `agentrun publish` to deploy to AgentCube
5. **Invocation**: Use `agentrun invoke` to test your deployed agent

```
┌─────────────┐     ┌──────────┐     ┌─────────┐     ┌─────────┐     ┌────────┐
│ Development │ --> │   Pack   │ --> │  Build  │ --> │ Publish │ --> │ Invoke │
│    Agent    │     │          │     │  Image  │     │   to    │     │ Agent  │
│    Code     │     │          │     │         │     │ AgentCube│    │        │
└─────────────┘     └──────────┘     └─────────┘     └─────────┘     └────────┘
```

## Prerequisites

Before starting this tutorial, ensure you have:

* **Python 3.8+** installed
* **Docker** installed and running (for local builds)
* **AgentRun CLI** installed
* Basic understanding of Python and HTTP APIs

## Step 1: Install AgentRun CLI

First, let's install the AgentRun CLI tool. You can install it from source or from PyPI (when available).

In [ ]:
# Install AgentRun CLI from the cli-agentrun directory
# Note: Adjust the path based on your setup
%pip install -e ../ --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Install agent dependencies
%pip install -r ./agent/requirements.txt --quiet

Verify the installation:

In [2]:
!agentrun --help

                                                                                
 Usage: agentrun [OPTIONS] COMMAND [ARGS]...                                    
                                                                                
 AgentRun CLI - A developer tool for packaging, building, and deploying AI      
 agents to AgentCube                                                            
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --version  -v        Show version and exit                                   │
│ --verbose            Enable verbose output                                   │
│ --help               Show this message and exit.                             │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ pack      Package the agen

**Expected Output:**

You should see the AgentRun CLI help message showing all available commands:
- `pack` - Package agent into standardized workspace
- `build` - Build container image
- `publish` - Publish agent to AgentCube
- `invoke` - Invoke published agent
- `status` - Check agent status

## Step 2: Explore the Example Agent

Let's examine the LangChain agent we'll be deploying. This agent provides a practical example of an AI agent that uses LangChain's framework with tool-calling capabilities.

In [3]:
# View the agent directory structure
!ls -la agent/

total 32
drwxr-xr-x  5 ym  staff   160 Nov 24 16:57 .
drwxr-xr-x  7 ym  staff   224 Nov 24 16:57 ..
-rw-r--r--  1 ym  staff   163 Nov 24 16:57 .env.example
-rw-r--r--  1 ym  staff  6374 Nov 24 16:57 main.py
-rw-r--r--  1 ym  staff   161 Nov 24 16:57 requirements.txt


**Expected Output:**

```
main.py          # The main agent code
requirements.txt # Python dependencies
```

In [4]:
# View a snippet of the agent code
!head -50 agent/main.py

"""
Simple LangChain Agent Application for Testing AgentRun CLI

This is a minimal agent application that demonstrates:
- LangChain agent with tools
- FastAPI web server
- JSON payload handling
- Ready for containerization with AgentRun CLI
"""

import os
import json
from typing import Optional, Dict, Any
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory


# Initialize FastAPI app
app = FastAPI(
    title="Simple LangChain Test Agent",
    description="A simple agent for testing AgentRun CLI pack and build commands",
    version="1.0.0"
)


# Request/Response models
class AgentRequest(BaseModel):
    prompt: str
    temperature: Optional[float] = 0.7
    max_tokens: Optional[int] = 500


class AgentResponse(BaseModel):
    response: str
    success: bool
    error: Optional[str] = None


# Simple calculator tool
d

### Understanding the Agent

The LangChain Agent provides four main endpoints:

1. **`GET /`** - Service information and health check
2. **`GET /health`** - Health check endpoint for container orchestration
3. **`GET /tools`** - List available tools (Calculator, Weather)
4. **`POST /invoke`** - Invoke the agent with a prompt

The agent uses LangChain framework and includes:
- **Calculator Tool**: Performs basic mathematical calculations (e.g., "2 + 2", "10 * 5")
- **Weather Tool**: Provides mock weather information for cities (Shanghai, Beijing, New York, London)
- **Mock Mode**: Works without OpenAI API key for testing purposes
- **Conversational Memory**: Maintains context across interactions
- **FastAPI**: Modern async web framework for HTTP endpoints

## Step 3: Test the Agent Locally (Optional)

Before deploying, you can test the agent locally to ensure it works correctly.

In [10]:
# Start the agent in the background (this cell will start a background process)
import subprocess
import time
import signal

# Start the agent
proc = subprocess.Popen(
    ['python', 'agent/main.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(2)

print("Agent started on port 8080")
print("PID:", proc.pid)

Agent started on port 8080
PID: 21297


In [11]:
# Test the agent with sample requests
import requests
import json

try:
    # Test health endpoint
    response = requests.get('http://localhost:8080/health')
    print("Health Check:")
    print(json.dumps(response.json(), indent=2))
    print()
    
    # Test tools listing
    response = requests.get('http://localhost:8080/tools')
    print("Available Tools:")
    print(json.dumps(response.json(), indent=2))
    print()
    
    # Test agent invocation with calculator
    response = requests.post(
        'http://localhost:8080/invoke',
        json={'prompt': 'What is 25 multiplied by 4?'}
    )
    print("Agent Response (Calculator Query):")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the agent is running on port 8080")

Health Check:
{
  "status": "healthy"
}

Available Tools:
{
  "tools": [
    {
      "name": "Calculator",
      "description": "Useful for performing mathematical calculations. Input should be a mathematical expression like '2 + 2' or '10 * 5'."
    },
    {
      "name": "Weather",
      "description": "Useful for getting weather information for a city. Input should be a city name like 'Shanghai' or 'New York'."
    }
  ]
}

Agent Response (Calculator Query):
{
  "response": "[Mock Response] Received prompt: 'What is 25 multiplied by 4?'. This would normally process your request using LangChain with OpenAI. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}


**Expected Output:**

```json
Health Check:
{
  "status": "healthy"
}

Available Tools:
{
  "tools": [
    {
      "name": "Calculator",
      "description": "Useful for performing mathematical calculations. Input should be a mathematical expression like '2 + 2' or '10 * 5'."
    },
    {
      "name": "Weather",
      "description": "Useful for getting weather information for a city. Input should be a city name like 'Shanghai' or 'New York'."
    }
  ]
}

Agent Response (Calculator Query):
{
  "response": "[Mock Response] Received prompt: 'What is 25 multiplied by 4?'. This would normally use the Calculator tool to perform the calculation. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

Note: Without an OpenAI API key, the agent runs in mock mode. With a valid API key, it will use LangChain to process requests and call the appropriate tools.

In [12]:
# Stop the local agent
try:
    proc.terminate()
    proc.wait(timeout=5)
    print("Agent stopped")
except:
    proc.kill()
    print("Agent killed")

Agent stopped


## Step 4: Package the Agent with `agentrun pack`

Now let's package our agent using the AgentRun CLI. The `pack` command:

- Validates the agent structure
- Creates `agent_metadata.yaml` configuration file
- Generates a `Dockerfile` automatically
- Prepares the workspace for building

In [13]:
# Package the agent
!agentrun pack \
    -f agent \
    --agent-name "langchain-agent" \
    --description "A LangChain agent with Calculator and Weather tools" \
    --language "python" \
    --entrypoint "python main.py" \
    --port 8080 \
    --build-mode "local" \
    --verbose

INFO:agentrun.runtime.pack_runtime:Starting pack process for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent
DEBUG:agentrun.runtime.pack_runtime:Workspace structure validation passed: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent
DEBUG:agentrun.runtime.pack_runtime:Creating new metadata
DEBUG:agentrun.services.metadata_service:Saving metadata to: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent/agent_metadata.yaml
DEBUG:agentrun.runtime.pack_runtime:Applied option overrides: {'agent_name': 
'langchain-agent', 'language': 'python', 'entrypoint': 'python main.py', 'port':
8080, 'build_mode': 'local'}
DEBUG:agentrun.runtime.pack_runtime:Found requirements.txt, processing 
dependencies
DEBUG:agentrun.runtime.pack_runtime:Generated Dockerfile: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent/Dockerfile
DEBUG:agentrun.services.meta

**Expected Output:**

```
✅ Successfully packaged agent: langchain-agent
📁 Workspace: /path/to/agent
📄 Metadata: /path/to/agent/agent_metadata.yaml
```

### Understanding the Generated Files

After packaging, let's examine the generated files:

In [14]:
# View the generated agent_metadata.yaml
!cat agent/agent_metadata.yaml

agent_name: langchain-agent
build_mode: local
description: A LangChain agent with Calculator and Weather tools
entrypoint: python main.py
language: python
port: 8080
requirements_file: requirements.txt


**Expected Output:**

```yaml
agent_name: langchain-agent
description: A LangChain agent with Calculator and Weather tools
language: python
entrypoint: python main.py
port: 8080
build_mode: local
requirements_file: requirements.txt
```

This metadata file is the central configuration for your agent. It defines:
- **agent_name**: Unique identifier for your agent
- **description**: Human-readable description
- **language**: Programming language (python, java)
- **entrypoint**: Command to start the agent
- **port**: Port the agent listens on
- **build_mode**: Build strategy (local or cloud)
- **requirements_file**: Dependencies file

In [15]:
# View the generated Dockerfile
!cat agent/Dockerfile

# Python Agent Dockerfile
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    gcc \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements first for better caching
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Expose port
EXPOSE 8080

# Set environment variables
ENV PYTHONPATH=/app
ENV PYTHONUNBUFFERED=1

# Run the application
CMD python main.py


**Expected Output:**

```dockerfile
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    gcc \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements first for better caching
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Expose port
EXPOSE 8080

# Set environment variables
ENV PYTHONPATH=/app
ENV PYTHONUNBUFFERED=1

# Run the application
CMD python main.py
```

The Dockerfile is automatically generated based on your agent's language and configuration.

## Step 5: Build the Agent Image with `agentrun build`

Now let's build a container image for our agent. The `build` command:

- Builds a Docker image from your agent workspace
- Supports local Docker builds and cloud builds
- Updates metadata with build information
- Provides detailed build progress

In [11]:
# Build the agent image
!agentrun build -f agent --verbose

DEBUG:docker.utils.config:Trying paths: ['/Users/ym/.docker/config.json', 
'/Users/ym/.dockercfg']
DEBUG:docker.utils.config:Found file at path: /Users/ym/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/ is absent. Client 
might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/access-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/refresh-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'credsStore' section
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/1.1" 200 
None
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.52/_ping HTTP/1.1" 
200 None
INFO:agentrun.services.docker_service:Successfully connected to Docker daemon
INFO:agentrun.runtime.build_runtime:Starting build process for workspace: 
/Users/ym/Desktop/a

**Expected Output:**

```
✅ Successfully built agent image: langchain-agent
🏷️  Tag: latest
📏 Size: ~200MB
```

The build process:
1. Reads the Dockerfile and agent_metadata.yaml
2. Builds the Docker image locally
3. Tags the image with the agent name
4. Updates metadata with build information

In [17]:
# Verify the Docker image was created
!docker images | grep langchain-agent

langchain-agent:latest                                                                                  e430f2bb9e60        523MB             0B        


**Expected Output:**

```
langchain-agent    latest    abc123def456    1 minute ago    200MB
```

## Step 6: Test the Built Container (Optional)

Before publishing to AgentCube, you can test the container locally to ensure it works correctly.

In [12]:
# Run the container locally
!docker run -d -p 8080:8080 --name langchain-agent-test langchain-agent:latest

import time
time.sleep(3)  # Wait for container to start
print("Container started!")

a9eda31e1deefd09f985e34928dd1f51e9463cf78e0cb1991923692a5ff14789
Container started!


In [13]:
# Test the containerized agent
import requests
import json

try:
    response = requests.post(
        'http://localhost:8080/',
        json={'prompt': 'What is the weather in Shanghai?'}
    )
    print("Containerized Agent Response:")
    print(json.dumps(response.json(), indent=2))
except Exception as e:
    print(f"Error: {e}")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:7897
DEBUG:urllib3.connectionpool:http://127.0.0.1:7897 "POST http://localhost:8080/ HTTP/1.1" 200 249


Containerized Agent Response:
{
  "response": "[Mock Response] Received prompt: 'What is the weather in Shanghai?'. This would normally use the Weather tool to fetch weather information. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}


**Expected Output:**

```json
{
  "response": "[Mock Response] Received prompt: 'What is the weather in Shanghai?'. This would normally use the Weather tool to fetch weather information. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

Note: The agent is running in mock mode. With a valid OpenAI API key, it would use the Weather tool to provide weather information.

In [14]:
# Stop and remove the test container
!docker stop langchain-agent-test
!docker rm langchain-agent-test

langchain-agent-test
langchain-agent-test


## Step 7: Publish the Agent with `agentrun publish`

Now let's publish our agent to AgentCube. The `publish` command:

- Registers the agent with AgentCube
- Pushes the Docker image to a registry (if using cloud mode)
- Creates a deployment endpoint
- Updates metadata with deployment information

**Note**: In the MVP version, the publish command simulates AgentCube integration for demonstration purposes.

In [15]:
# Publish the agent to AgentCube
!agentrun publish \
    -f agent \
    --version "v1.0.0" \
    --image-url "taoruiw/langchain-agent:latest" \
    --description "LangChain agent with Calculator and Weather tools" \
    --verbose \
    --use-k8s

DEBUG:docker.utils.config:Trying paths: ['/Users/ym/.docker/config.json', 
'/Users/ym/.dockercfg']
DEBUG:docker.utils.config:Found file at path: /Users/ym/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/ is absent. Client 
might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/access-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/refresh-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'credsStore' section
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/1.1" 200 
None
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.52/_ping HTTP/1.1" 
200 None
INFO:agentrun.services.docker_service:Successfully connected to Docker daemon
INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_

**Expected Output:**

```
✅ Successfully published agent: langchain-agent
🆔 Agent ID: agent-abc123
🌐 Endpoint: https://api.agentcube.example.com/agents/agent-abc123/invoke
```

### Understanding the Publish Process

The publish command:
1. Validates that the agent has been built
2. Tags and pushes the image to the specified registry
3. Registers the agent with AgentCube API
4. Receives an agent ID and endpoint URL
5. Updates the metadata with deployment information

In [23]:
# View the updated metadata after publishing
!cat agent/agent_metadata.yaml

agent_endpoint: http://localhost:31941
agent_id: langchain-agent
agent_name: langchain-agent
build_mode: local
description: A LangChain agent with Calculator and Weather tools
entrypoint: python main.py
image:
  build_mode: local
  build_size: 498.8MB
  build_time: 32.9s
  repository_url: langchain-agent:latest
  tag: latest
k8s_deployment:
  container_port: 8080
  deployment_name: langchain-agent
  namespace: agentrun
  node_port: 31941
  replicas: 1
  service_name: langchain-agent
  service_url: http://localhost:31941
language: python
port: 8080
requirements_file: requirements.txt


**Expected Output:**

```yaml
agent_name: langchain-agent
description: LangChain agent with Calculator and Weather tools
language: python
entrypoint: python main.py
port: 8080
build_mode: local
requirements_file: requirements.txt
version: v1.0.0
agent_id: agent-abc123
agent_endpoint: https://api.agentcube.example.com/agents/agent-abc123/invoke
image:
  repository_url: docker.io/myorg/langchain-agent
  tag: latest
  build_mode: local
```

## Step 8: Check Agent Status with `agentrun status`

You can check the status of your published agent at any time using the `status` command.

In [16]:
# Check agent status
!agentrun status -f agent --verbose --use-k8s

INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_provider:Kubernetes provider initialized for namespace: agentrun
DEBUG:kubernetes.client.rest:response body: {"kind":"Namespace","apiVersion":"v1","metadata":{"name":"agentrun","uid":"40e8bdbd-a04c-4884-b7ea-7883ef65df9b","resourceVersion":"16799","creationTimestamp":"2025-11-18T12:02:51Z","labels":{"kubernetes.io/metadata.name":"agentrun"},"managedFields":[{"manager":"OpenAPI-Generator","operation":"Update","apiVersion":"v1","time":"2025-11-18T12:02:51Z","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":{"f:labels":{".":{},"f:kubernetes.io/metadata.name":{}}}}}]},"spec":{"finalizers":["kubernetes"]},"status":{"phase":"Active"}}

DEBUG:agentrun.services.k8s_provider:Namespace agentrun already exists
INFO:agentrun.runtime.status_runtime:Checking agent status for workspace: /Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/agent
DEBUG:agentrun.services.metadata_service

**Expected Output:**

```
╭───────────────────────────────────────────────────────────────╮
│                       Agent Status                             │
├────────────┬──────────────────────────────────────────────────┤
│ Property   │ Value                                             │
├────────────┼──────────────────────────────────────────────────┤
│ Agent Name │ langchain-agent                                   │
│ Agent ID   │ agent-abc123                                      │
│ Version    │ v1.0.0                                            │
│ Language   │ python                                            │
│ Build Mode │ local                                             │
│ Endpoint   │ https://api.agentcube.example.com/agents/...     │
╰────────────┴──────────────────────────────────────────────────╯
```

## Step 9: Invoke the Published Agent with `agentrun invoke`

Finally, let's test our published agent by invoking it with a sample payload.

In [17]:
!agentrun invoke --help

                                                                                
 Usage: agentrun invoke [OPTIONS]                                               
                                                                                
 Invoke a published agent via AgentCube or Kubernetes.                          
                                                                                
 This command sends a request to a published agent, allowing you                
 to test and interact with your deployed agent.                                 
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --workspace  -f      TEXT  Path to the agent workspace directory             │
│                            [default: .]                                      │
│ --payload            TEXT  JSON-formatted input passed to the agent          │
│                           

In [18]:
# Invoke the agent with a calculator query
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "What is 15 plus 27?"}' \
    --verbose

INFO:agentrun.runtime.invoke_runtime:Starting agent invocation for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent
DEBUG:agentrun.services.metadata_service:Loading metadata from: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent/agent_metadata.yaml
DEBUG:agentrun.runtime.invoke_runtime:Invocation prerequisites validated: 
agent_id=langchain-agent
DEBUG:asyncio:Using selector: KqueueSelector
INFO:agentrun.runtime.invoke_runtime:Invoking agent langchain-agent at 
http://localhost:31283
INFO:agentrun.runtime.invoke_runtime:Making HTTP POST request to 
http://localhost:31283
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=7897 
local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete 
return_value=<httpcore._backends.anyio.AnyIOStream object at 0x104359350>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:http

**Expected Output:**

```
✅ Successfully invoked agent
📤 Response: {
  "response": "[Mock Response] Received prompt: 'What is 15 plus 27?'. This would normally use the Calculator tool to perform the calculation. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

With a valid OpenAI API key, the response would be:
```
✅ Successfully invoked agent
📤 Response: {
  "response": "The result of 15 plus 27 is 42.",
  "success": true,
  "error": null
}
```

In [19]:
# Invoke with a weather query
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "What is the weather like in Beijing?"}' \
    --verbose

INFO:agentrun.runtime.invoke_runtime:Starting agent invocation for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent
DEBUG:agentrun.services.metadata_service:Loading metadata from: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent/agent_metadata.yaml
DEBUG:agentrun.runtime.invoke_runtime:Invocation prerequisites validated: 
agent_id=langchain-agent
DEBUG:asyncio:Using selector: KqueueSelector
INFO:agentrun.runtime.invoke_runtime:Invoking agent langchain-agent at 
http://localhost:31283
INFO:agentrun.runtime.invoke_runtime:Making HTTP POST request to 
http://localhost:31283
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=7897 
local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete 
return_value=<httpcore._backends.anyio.AnyIOStream object at 0x1069f9090>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:http

**Expected Output:**

```
✅ Successfully invoked agent
📤 Response: {
  "response": "[Mock Response] Received prompt: 'What is the weather like in Beijing?'. This would normally use the Weather tool to fetch weather information. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

With a valid OpenAI API key, the agent would use the Weather tool and respond:
```
✅ Successfully invoked agent
📤 Response: {
  "response": "The weather in Beijing is Cloudy, 18°C, with 45% humidity.",
  "success": true,
  "error": null
}
```

In [20]:
# Invoke with a general query
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "Tell me about the available tools."}' \
    --verbose

INFO:agentrun.runtime.invoke_runtime:Starting agent invocation for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent
DEBUG:agentrun.services.metadata_service:Loading metadata from: 
/Users/ym/Desktop/agentcube/cmd/agentrun/01-getting-started-with-agentrun-cli/ag
ent/agent_metadata.yaml
DEBUG:agentrun.runtime.invoke_runtime:Invocation prerequisites validated: 
agent_id=langchain-agent
DEBUG:asyncio:Using selector: KqueueSelector
INFO:agentrun.runtime.invoke_runtime:Invoking agent langchain-agent at 
http://localhost:31283
INFO:agentrun.runtime.invoke_runtime:Making HTTP POST request to 
http://localhost:31283
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=7897 
local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete 
return_value=<httpcore._backends.anyio.AnyIOStream object at 0x1083f36d0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:http

**Expected Output:**

```
✅ Successfully invoked agent
📤 Response: {
  "response": "[Mock Response] Received prompt: 'Tell me about the available tools.'. This would normally process your request using LangChain with OpenAI. (Set OPENAI_API_KEY environment variable for real agent responses)",
  "success": true,
  "error": null
}
```

With a valid OpenAI API key, the agent would provide information about its tools:
```
✅ Successfully invoked agent
📤 Response: {
  "response": "I have access to two tools: a Calculator for performing mathematical calculations, and a Weather tool for getting weather information for various cities.",
  "success": true,
  "error": null
}
```

## Summary and Best Practices

### What We Learned

In this tutorial, we covered the complete AgentRun CLI workflow:

1. ✅ **Installed** AgentRun CLI
2. ✅ **Explored** a practical sentiment analysis agent
3. ✅ **Tested** the agent locally
4. ✅ **Packaged** the agent with `agentrun pack`
5. ✅ **Built** a container image with `agentrun build`
6. ✅ **Published** the agent with `agentrun publish`
7. ✅ **Checked** status with `agentrun status`
8. ✅ **Invoked** the agent with `agentrun invoke`

### Best Practices

#### 1. Project Structure
```
my-agent/
├── main.py              # Agent entrypoint
├── requirements.txt     # Dependencies
├── README.md           # Documentation
├── agent_metadata.yaml # Generated by agentrun pack
└── Dockerfile          # Generated by agentrun pack
```

#### 2. Agent Development
- **Test locally first** before packaging and deploying
- **Implement health checks** at `/health` endpoint
- **Support both `text` and `prompt`** fields for flexibility
- **Return structured JSON** responses
- **Handle errors gracefully** with proper HTTP status codes
- **Add logging** for debugging and monitoring

#### 3. Metadata Configuration
- **Use descriptive names** for your agents
- **Provide clear descriptions** to help users understand your agent
- **Specify correct entrypoint** and port
- **Choose appropriate build mode** (local for development, cloud for production)
- **Use semantic versioning** for version numbers (v1.0.0, v1.1.0, etc.)

#### 4. Building and Deployment
- **Test container locally** before publishing
- **Use proper image naming** conventions (organization/agent-name)
- **Tag images with versions** for easier rollback
- **Keep images small** by using slim base images
- **Document dependencies** clearly in requirements.txt

#### 5. CI/CD Integration
- **Use Python SDK** for automation
- **Implement automated testing** before deployment
- **Version control** your agent code and metadata
- **Use environment variables** for configuration
- **Set up monitoring** for production agents

### Command Reference

| Command | Purpose | Example |
|---------|---------|----------|
| `agentrun pack` | Package agent workspace | `agentrun pack -f ./agent --agent-name my-agent` |
| `agentrun build` | Build container image | `agentrun build -f ./agent --verbose` |
| `agentrun publish` | Publish to AgentCube | `agentrun publish -f ./agent --version v1.0.0` |
| `agentrun invoke` | Test deployed agent | `agentrun invoke -f ./agent --payload '{"text": "test"}'` |
| `agentrun status` | Check agent status | `agentrun status -f ./agent` |

### Next Steps

Now that you understand the basics, you can:

1. **Enhance the sentiment agent** with more sophisticated NLP capabilities
2. **Add authentication** using custom headers
3. **Implement streaming responses** for long-running tasks
4. **Create multi-agent systems** with agent-to-agent communication
5. **Integrate with external APIs** and services
6. **Build custom agents** for your specific use cases
7. **Explore cloud build mode** for serverless deployments
8. **Set up monitoring and logging** for production agents

### Additional Resources

- [AgentRun CLI Documentation](../../README.md)
- [Quick Start Guide](../../QUICKSTART.md)
- [Example Agents](../../examples/)
- [AgentCube Main Project](https://github.com/volcano-sh/agentcube)

### Getting Help

If you encounter issues or have questions:

- Run commands with `--verbose` flag for detailed logging
- Check `agentrun <command> --help` for command-specific help
- Review the [troubleshooting guide](../../QUICKSTART.md#troubleshooting)
- Open an issue on [GitHub](https://github.com/volcano-sh/agentcube/issues)

## Cleanup (Optional)

If you want to clean up the resources created in this tutorial:

In [ ]:
# Remove Docker images
!docker rmi langchain-agent:latest

In [ ]:
# Remove generated files (optional)
!rm -f agent/agent_metadata.yaml
!rm -f agent/Dockerfile

# Congratulations! 🎉

You've successfully completed the AgentRun CLI tutorial and learned how to develop, package, build, publish, and invoke AI agents using AgentRun CLI!

Happy agent building! 🚀